In [1]:
import keras
from keras.applications.resnet import ResNet50
from keras.applications.resnet import preprocess_input, decode_predictions

from keras.applications.resnet import ResNet101
from keras.preprocessing import image

import numpy as np
from collections import defaultdict
from tqdm.notebook import tqdm, trange

2024-10-01 15:47:31.236239: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-01 15:47:31.379804: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# instantiate ResNet50 model
model50 = ResNet50(weights='imagenet')

# instantiate ResNet101 model
model101 = ResNet101(weights='imagenet')

2024-10-01 15:47:33.009913: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-01 15:47:33.086344: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-01 15:47:33.088544: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [3]:
IMG_PATH = 'bernie.png'
LABELS_PATH = 'labels.txt'
SIDE = 224
ALPHA = 0.5

with open(LABELS_PATH) as f:
    labels = f.read()
    labels = eval(labels)

In [4]:
resnet101_target = 'printer'
resnet50_target = 'gazzelle'

In [5]:
base_img = image.load_img(IMG_PATH)
base_img = image.img_to_array(base_img)
assert base_img.shape == (SIDE, SIDE, 3)

In [6]:
def get_rnd_noise_img():
    return np.random.uniform(0, 255, (SIDE, SIDE, 3))

def mix_images(img1, img2, alpha):
    return alpha * img1 + (1 - alpha) * img2

In [ ]:
count = defaultdict(int)

for i in trange(10000):
    noise = get_rnd_noise_img()
    # img = mix_images(base_img, noise, ALPHA)
    img = noise
    pred50 = model50.predict(preprocess_input(img[None]))
    pred101 = model101.predict(preprocess_input(img[None]))
    pred50 = decode_predictions(pred50, top=1)[0][0][1].replace('_', ' ')
    pred101 = decode_predictions(pred101, top=1)[0][0][1].replace('_', ' ')
    count[pred50] += 1
    count[pred101] += 1
    if pred101 == resnet101_target:
        print(pred101)
        image.save_img('found101.png', img)
    if pred50 == resnet50_target:
        print(pred50)
        image.save_img('found50.png', img)
    if pred50 == resnet50_target and pred101 == resnet101_target:
        # save to file
        print('Found image')
        image.save_img('found.png', img)
        break

  0%|          | 0/10000 [00:00<?, ?it/s]

2024-10-01 15:48:00.074041: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x46ce7c10 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-10-01 15:48:00.074085: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9
2024-10-01 15:48:00.151123: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-10-01 15:48:00.557234: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2024-10-01 15:48:00.685793: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step


2024-10-01 15:48:02.339151: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step

In [8]:
print(count)

defaultdict(<class 'int'>, {'chain mail': 1985, 'assault rifle': 364, 'snorkel': 2374, 'sunglasses': 1821, 'gasmask': 1064, 'stingray': 212, 'mask': 1425, 'ski mask': 224, 'window screen': 107, 'loupe': 33, 'feather boa': 69, 'strainer': 10, 'trench coat': 51, 'cuirass': 6, 'miniature poodle': 9, 'miniature pinscher': 37, 'shovel': 15, 'cocker spaniel': 53, 'schipperke': 6, 'soft-coated wheaten terrier': 3, 'American black bear': 18, 'loudspeaker': 42, 'Staffordshire bullterrier': 4, 'oxygen mask': 13, 'cowboy hat': 2, 'bow tie': 6, 'laptop': 1, 'Scotch terrier': 7, 'miniature schnauzer': 2, 'car mirror': 16, 'Chihuahua': 7, 'toy poodle': 1, 'American Staffordshire terrier': 1, 'sweatshirt': 2, 'bulletproof vest': 3, 'puffer': 5, 'French bulldog': 1, 'breastplate': 1})


defaultdict(<class 'int'>, {'tennis ball': 1433, 'window screen': 1883, 'chainlink fence': 314, 'chain': 63, 'brain coral': 19, 'coral reef': 44, 'chain mail': 3, 'shower curtain': 1})
